# Passing Networks
Visualization and calculation the centralized nature of a game for teams involved.  Normally this is evaluated for one of the teams in a given game.  This helps to show the average shape of the team throughout the game.  Normally stopped at the first substution.

## Imports
This version will deviate from the Soccermatics approach due to the mplsoccer library throwing an error when making a call to Statsbomb.  Instead, the statsbomb data will be called directly for the dataframe from the statsbomb library.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mplsoccer import Pitch, Sbopen, VerticalPitch
from statsbombpy import sb

## Opening and creating the dataset
Using the sb parser in place of SBopen in mplsoccer the event data is pulled out. Calling on the event level data and inputting the id of the game as a parameter loads the game data. The event data is stored in a dataframe df. From this dataframe, the next step will be to filter for passes not including throw ins are left in the frame.

In [5]:
# get event data
df = sb.events(match_id=69301)

In [6]:
df.head(2)

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4141, 'lineup': [{'player': {'id...",England Women's,865,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Sweden Women's,858,00:00:00.000,Starting XI,NaN


## Preparing the data
For passing networks only use accurate/successful passes made by a team until the first substitution. This is the most basic use case, and there are several possible variations using passing networks. They require information about pass start and end location as well as player who made and received the pass. To make the vizualisation clearer annotate the players using their surname. 

In [8]:
# find index of first sub for England
sub = df.loc[df["type"] == "Substitution"].loc[df["team"] == "England Women's"].iloc[0]["index"]
sub

1764

In [ ]:
#make df with successfull passes by England until the first substitution
mask_england = (df.type_name == 'Pass') & (df.team_name == "England Women's") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")


(df
 .query('Pass=="Pass"')
)

#taking necessary columns
df_pass = df.loc[mask_england, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
#adjusting that only the surname of a player is presented.
df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])